In [ ]:
from google.colab import drive
drive.mount('/content/drive')

- Shall we use Twitter dataset? because we can consider each tweet and its replies as a dialogue. but the problem is we do not have multiparty conversation anymore.

In [ ]:
import pandas as pd
import pickle
import json
from typing import List, Dict, Any
from collections import defaultdict

## **Read File**

In [ ]:
def read_jsonl_file(file_path: str) -> List[Dict[str, Any]]:
  data: List[Dict[str, Any]] = []
  with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
      data.append(json.loads(line))
  return data

**Wikipedia data**

In [ ]:
wikipedia = read_jsonl_file("/content/drive/MyDrive/ResponseGeneration/Original_Data/wikipedia.jsonl")

In [ ]:
wikipedia

## **Converting Original Datasets to prefered format.**

In [ ]:
def data_transformation(dataset: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    for utt in dataset:
        if 'reply-to' in utt:
            utt['reply_to'] = utt.pop('reply-to')

    filtered_utt: List[Dict[str, Any]] = []
    for utt in dataset:
        filtered_utt.append({key: utt[key] for key in ["id","user",'root', "reply_to", 'text', "timestamp"]})

    conversations: defaultdict[str, List[Dict[str, Any]]] = defaultdict(list)
    for utt in filtered_utt:
        root = utt["root"]
        conversations[root].append(utt)

    for id, conversation in conversations.items():
        # print(id)
        for utterance in conversation:
            if utterance.get('reply_to') is not None:
                try:
                    utterance['reply_to'] = int(utterance['reply_to'])
                except ValueError:
                    utterance['reply_to'] = utterance['reply_to']
            if utterance.get('id') == utterance.get('root'):
                pass
            elif utterance.get('reply_to') is None:
                try:
                    utterance['reply_to'] = int(utterance['id']) - 1
                except ValueError:
                    utterance['reply_to'] = utterance['root']

    print(f"We have {len(conversations)} conversations in this dataset")

    return conversations


In [ ]:
wikipedia_dataset = data_transformation(wikipedia)

In [ ]:
wikipedia_dataset

In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are my close friend who speaks so friendly with me"},
    {"role": "user", "content": "How are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])